In [1]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [2]:
import fitz

In [3]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [4]:
with fitz.open("paper1.pdf") as doc:
    ARTICLE = ""
    for page in doc:
        ARTICLE += page.get_text()

In [5]:
ARTICLE

'2022 IEEE India Council International Subsections Conference (INDISCON) \n \n978-1-6654-6601-1/22/$31.00 ©2022 IEEE \nPredictor Analysis and Proliferation of Fertility and \nProduction for Agriculturalists \n \nC.Shyamala Kumari \nComputer Science and Engineering,  \nVel Tech Rangarajan Dr.Sagunthala \nR&D Institute of Science and \nTechnology \nChennai, India \nshyamalakumaric@veltech.edu.in \nShourjya Hazra  \nComputer Science and Engineering,  \nVel Tech Rangarajan Dr.Sagunthala \nR&D Institute of Science and \nTechnology \nChennai, India \nshourjya001@gmail.com \nRohit Kumar \nComputer Science and Engineering, \nVel Tech Rangarajan Dr.Sagunthala \nR&D Institute of Science and \nTechnology  \nChennai, India \nrkrohit14092@gmail.com \nMayurakshya Paul \nComputer Science and Engineering, \nVel Tech Rangarajan Dr.Sagunthala \nR&D Institute of Science and \nTechnology \nChennai, India \nmayurakshyapaul@gmail.com\nSaurav Kumar Gupta \nComputer Science and Engineering,  \nVel Tech Rangar

In [6]:
max_chunk = 500

In [7]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')
ARTICLE = ARTICLE.replace('\n',' ')

In [8]:
ARTICLE

'2022 IEEE India Council International Subsections Conference (INDISCON)    978-1-6654-6601-1/22/$31.<eos>00 ©2022 IEEE  Predictor Analysis and Proliferation of Fertility and  Production for Agriculturalists    C.<eos>Shyamala Kumari  Computer Science and Engineering,   Vel Tech Rangarajan Dr.<eos>Sagunthala  R&D Institute of Science and  Technology  Chennai, India  shyamalakumaric@veltech.<eos>edu.<eos>in  Shourjya Hazra   Computer Science and Engineering,   Vel Tech Rangarajan Dr.<eos>Sagunthala  R&D Institute of Science and  Technology  Chennai, India  shourjya001@gmail.<eos>com  Rohit Kumar  Computer Science and Engineering,  Vel Tech Rangarajan Dr.<eos>Sagunthala  R&D Institute of Science and  Technology   Chennai, India  rkrohit14092@gmail.<eos>com  Mayurakshya Paul  Computer Science and Engineering,  Vel Tech Rangarajan Dr.<eos>Sagunthala  R&D Institute of Science and  Technology  Chennai, India  mayurakshyapaul@gmail.<eos>com Saurav Kumar Gupta  Computer Science and Engineering

In [9]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [11]:
sentences

['2022 IEEE India Council International Subsections Conference (INDISCON)    978-1-6654-6601-1/22/$31.',
 '00 ©2022 IEEE  Predictor Analysis and Proliferation of Fertility and  Production for Agriculturalists    C.',
 'Shyamala Kumari  Computer Science and Engineering,   Vel Tech Rangarajan Dr.',
 'Sagunthala  R&D Institute of Science and  Technology  Chennai, India  shyamalakumaric@veltech.',
 'edu.',
 'in  Shourjya Hazra   Computer Science and Engineering,   Vel Tech Rangarajan Dr.',
 'Sagunthala  R&D Institute of Science and  Technology  Chennai, India  shourjya001@gmail.',
 'com  Rohit Kumar  Computer Science and Engineering,  Vel Tech Rangarajan Dr.',
 'Sagunthala  R&D Institute of Science and  Technology   Chennai, India  rkrohit14092@gmail.',
 'com  Mayurakshya Paul  Computer Science and Engineering,  Vel Tech Rangarajan Dr.',
 'Sagunthala  R&D Institute of Science and  Technology  Chennai, India  mayurakshyapaul@gmail.',
 'com Saurav Kumar Gupta  Computer Science and Engineerin

In [12]:
len(chunks)

8

In [13]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)


In [14]:
res

[{'summary_text': ' About 60% of total agriculturalists in India are poor and can’t afford heavy robotics to take  advantage of the technology . Farmers are sometimes oblivious of the disease in the crop and the market prices of the products . A multilingual platform has been proposed which can  be accessed by all people and from where the farmers can easily  get the current price of their crops in the market .'},
 {'summary_text': ' A major part of the farming community in India is either unaware of these  technologies or they aren’t able to afford them due to immense  prices . The enhancement in the field of machine learning has  brought a revolution in overall surveillance and production . The objective of the paper is to make the  farming community aware of technology advancements and  make heavy technology available to them on an easy platform .'},
 {'summary_text': ' Machine-learning algorithms are performed on Python 3. 7 (Jupyter Notebook) using inbuilt libraries: Numpy, Pandas

In [15]:
' '.join([summ['summary_text'] for summ in res])

' About 60% of total agriculturalists in India are poor and can’t afford heavy robotics to take  advantage of the technology . Farmers are sometimes oblivious of the disease in the crop and the market prices of the products . A multilingual platform has been proposed which can  be accessed by all people and from where the farmers can easily  get the current price of their crops in the market .  A major part of the farming community in India is either unaware of these  technologies or they aren’t able to afford them due to immense  prices . The enhancement in the field of machine learning has  brought a revolution in overall surveillance and production . The objective of the paper is to make the  farming community aware of technology advancements and  make heavy technology available to them on an easy platform .  Machine-learning algorithms are performed on Python 3. 7 (Jupyter Notebook) using inbuilt libraries: Numpy, Pandas, Scikit-learn built on tensorflow backend . KNN is a supervis

In [16]:
text = ' '.join([summ['summary_text'] for summ in res])

In [17]:
with open('pap4.txt', 'w') as f:
    f.write(text)